In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r"D:\Stuff\CyberSec\Datasets\IDS2018\02-14-2018.csv")


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2)


In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Load the dataset into a pandas dataframe
data = pd.read_csv(r"D:\Stuff\CyberSec\Datasets\IDS2018\02-14-2018.csv")

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Define the input shape of the model
input_shape = (train_data.shape[1]-1,)

# Define the base model
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)

# Freeze the weights of the base model
base_model.trainable = False

# Create the final model by adding a few layers on top of the base model
inputs = keras.Input(shape=input_shape)
x = layers.Dense(128, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

# Flatten the output of the last Dense layer
x = layers.Flatten()(x)

# Compute the shape of the output of the ResNet50 base model
base_output_shape = base_model.layers[-1].output_shape[1:]

# Reshape the output of the last Dense layer to match the input shape of the base model
x = layers.Dense(tf.reduce_prod(base_output_shape))(x)
x = layers.Reshape(base_output_shape)(x)

# Apply the base model to the reshaped output of the last Dense layer
x = base_model(x, training=False)

# Add a few more layers on top of the base model
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

# Add the final output layer
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the final model
model = keras.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_data.iloc[:,:-1], train_data.iloc[:,-1],
    validation_split=0.2,
    epochs=10,
    batch_size=64
)

# Save the pretrained model
model.save('model')


ValueError: Exception encountered when calling layer "resnet50" (type Functional).

Input 0 of layer "conv1_conv" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 13, 13, 2048)

Call arguments received by layer "resnet50" (type Functional):
  • inputs=tf.Tensor(shape=(None, 7, 7, 2048), dtype=float32)
  • training=False
  • mask=None